In [1]:
import pandas as pd
import numpy as np
import re

# Pre-processamento
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, GridSearchCV

# Classificadores
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression

from warnings import filterwarnings

# Ignorando warnings de deprecation
filterwarnings(action='ignore')

In [2]:
data = pd.read_csv('data/data_estag_ds.tsv', sep='\t')

In [3]:
data.head(10)

,ID,TITLE
0,1041354,Acessório T - Jean Bag For Girls para DS Lite
1,1041782,Carrinho de Bebê Berço-Passeio - Pegasus Pink ...
2,1041834,Carrinho de Bebê para Gêmeos Berço-Passeio - T...
3,1042568,Car Center - Calesita
4,1042584,Donka Trem com Som - Calesita
5,1043696,Interfone HDL AZ-01 Branco
6,1940486,Smartphone LG Optimus L7 II Dual P716 Preto co...
7,2277815,Smartphone Samsung Galaxy S4 Mini Duos Preto c...
8,2961097,Smartphone Samsung Galaxy S5 SM-G900M Branco ...
9,2961099,Smartphone Samsung Galaxy S5 SM-G900M Dourado...


In [4]:
data.shape

(2606, 2)

In [5]:
# Removendo o atributo ID (irrelevante)
ids = data['ID']
data.drop(['ID'], axis=1, inplace=True)

In [6]:
# Inicializando atributos target com 0.0
data['SMARTPHONE'] = np.zeros((len(data)))

# Parte 1: Anotação dos dados
Objetivando a aplicação de um algoritmo supervisionado, foram classificados manualmente os dados entre 1.0 (Smartphone) e 0.0 (Não-Smartphone) e depois filtrados os possíveis casos indesejáveis. Tal tarefa é relativamente simples, pois os intervalos onde há smartphones são facilmente identificáveis. Caso os dados estivessem muito embaralhados e/ou o conjunto de dados fosse muito grande, outras técnicas poderiam ser usadas (i.e. clustering, regex).

In [7]:
# Manualmente selecionando os intervalos onde existem smartphones
data['SMARTPHONE'].iloc[6:39] = 1.0
data['SMARTPHONE'].iloc[2035:] = 1.0

# Nota: Há alguns smartwatches nesse intervalo.
#       Além disso, pode ser que haja algum smartphone que não esteja nesse intervalo.
#       Esses casos serão tratados a seguir

In [8]:
# Casos de Smartwatches classificados como 1.0
for i, example in data[data['SMARTPHONE'] == 1.0]['TITLE'].iteritems():
    if re.findall('watch|rel[ó|o]gio', example.lower()):
        print(i, example)
        data['SMARTPHONE'].iloc[i] = 0.0

2207 Apple Watch Série 3 42mm Cores E Edição  Nike+ Em Estoque
2227 Apple Watch Series 3 42mm Nike + Gps Prova D'água Lacrado
2244 Apple Watch Series 3 42mm Gps Garantia Apple + Brinde
2253 Relógio Samsung Gear S3 Sm-r770 Classic Wifi Bluetooth
2288 Apple Watch Series 3  42mm Nike + Gps Prova D'água Lançameto
2289 Apple Watch Series 3 42mm Nike +  Gps Prova D'água
2298 Apple Watch Series 3 38mm Nike + Gps Prova D'água
2301 Apple Watch Nike+ Séries 3 42mm Lacrado
2302 Apple Watch Nike+ Séries 3 38mm Lacrado Novo
2303 Apple Watch Nike+ Séries 3 38mm Lacrado
2309 Apple Watch Series 3 38mm Nike + Gps Prova D'água Lacrado
2314 Apple Watch Série 3 42mm Gps + 2 Pulseiras Extras
2333 Relógio Samsung S3 Frontier + 2 Pulseiras Extras
2345 Apple Watch Series 3 42mm Lacrado + Brinde
2353 Apple Watch Nike+ Séries 3 42mm Lacrado + Envio Imediato
2380 Apple Watch Series 3 38mm Lacrado Nf + Brinde
2417 Smartwatch Samsung Gear S3 Frontier 1.3 Bluetooth - Original
2597 Relógio Smartwatch Samsung Gear Sm

In [9]:
# Casos de smartphones classificados como 0.0
for i, example in data[data['SMARTPHONE'] == 0.0]['TITLE'].iteritems():
    if re.findall('^smartphone|^celular', example.lower()):
        print(i, example)
        
# Somente 2 casos
data['SMARTPHONE'].iloc[1892:1894] = 1.0

# Nota: Foram encontrados alguns celulares, mas que não são smartphones. Estes foram desconsiderados.

548 Celular Multilaser Up- Branco/Rosa
775 Celular Blu Samba Jr Q53 Prata Dual Chip Silver
1886 Celular Up 2chip Quad Cam Mp3/4 Fm Branco/Rosa
1892 Smartphone Samsung Galaxy J2 Prime TV Dourado com 8GB, Dual chip, Tela 5", TV Digital, 4G, Câmera 8MP, Android 6.0 e Processador Quad Core de 1.4 Ghz
1893 Smartphone Samsung Galaxy J2 Prime TV Preto com 8GB, Dual chip, Tela 5", TV Digital, 4G, Câmera 8MP, Android 6.0 e Processador Quad Core de 1.4 Ghz


In [10]:
data['SMARTPHONE'].value_counts()

0.0    2018
1.0     588
Name: SMARTPHONE, dtype: int64

In [11]:
data['SMARTPHONE'].mean()
# Classes estão desbalanceadas

0.2256331542594014

# Parte 2: Pré-processamento
Nessa etapa serão feitos processos para normalizar os dados. Serão removidas stopwords, todas as palavras serão transformadas em representações de letras minúsculas e então será feita tokenização e bag of words.

In [12]:
# Adicionando pontuações às stopwords em português
pontuacoes = ['-', '.', ',', '(', ')']
pt_stopwords = stopwords.words('portuguese')

for pontuacao in pontuacoes:
    pt_stopwords.append(pontuacao)

In [13]:
# CountVectorizer remove os stopwords, tokeniza e obtem bag of words
vect = CountVectorizer(stop_words=pt_stopwords)
vect = CountVectorizer().fit(data['TITLE'])

data_vect = vect.transform(data['TITLE'])

In [14]:
data_vect

<2606x4165 sparse matrix of type '<class 'numpy.int64'>'
	with 21947 stored elements in Compressed Sparse Row format>

# Parte 3: Seleção do Modelo e Validação
Será utilizado Grid Search para encontrar os melhores parâmetros para cada classificador. O GS tem 5 folds de cross validation, método de avaliação f1 (pois as classes estão desbalanceadas) e é feita paralelização em 4 threads. Os classificadores escolhidos para a tarefa foram:

### SVM:
- Funciona bem em dados com alta dimensão e poucos exemplos (dataset está com 2606 linhas e 4165 colunas);
- Variando parâmetros C e gamma.

### Regressão Logística:
- Modelo simples e eficiente;
- Variando penalidades l1 e l2 e o parâmetro C.

### Bernoulli:
- Naive Bayes de representação binária;
- Variando constante de suavização de laplace.

### Multinomial:
- Naive Bayes para representação de múltiplas ocorrências;
- Variando constante de suavização de laplace.

In [15]:
# Definindo os parâmetros a serem selecionados para cada classificador
SVM = {'name': 'svm',
       'classifier': SVC(),
       'parameters': {'kernel': ['linear'],
       'C': [0.1, 1.0, 5, 10, 15],
       'gamma': [0.01, 0.1, 1.0, 5.0, 10],}}

Logistic = {'name': 'logistic_regression',
       'classifier': LogisticRegression(),
       'parameters': {'penalty': ['l1', 'l2'],
                      'C': [0.01, 0.1, 1.0, 5.0, 10],
                      'solver': ['liblinear']}}

Bernoulli = {'name': 'bernoulli_naive_bayes',
             'classifier': BernoulliNB(),
             'parameters': {'alpha': [1.0, 2.0, 5.0]}}
            
Multinomial = {'name': 'multinomial_naive_bayes',
               'classifier': MultinomialNB(),
               'parameters': {'alpha': [1.0, 2.0, 5.0]}}

classificadores = [SVM, Logistic, Bernoulli, Multinomial]

In [16]:
# Encontra os melhores parâmetros para cada classificador

modelos = {}
for classificador in classificadores:
    gs = GridSearchCV(estimator=classificador['classifier'],
                                param_grid=classificador['parameters'],
                                scoring='f1',
                                verbose=0,
                                cv=5,
                                n_jobs=4)

    gs.fit(data_vect, data['SMARTPHONE'])
    modelos[classificador['name']] = gs.best_estimator_

In [17]:
# Avaliação usando cross validation score
resultados = {}
for nome, modelo in modelos.items():
    resultados[nome] = np.mean(cross_val_score(modelo,
                                            data_vect,
                                            data['SMARTPHONE'],
                                            cv=5,
                                            scoring='f1'))
    
for nome, resultado in resultados.items():
    print(nome+':', resultado)
    
# Regressor Logístico > SVM > BernoulliNB > MultinomialNB

svm: 0.9820169917387778
logistic_regression: 0.9835542996094679
bernoulli_naive_bayes: 0.9485674493873768
multinomial_naive_bayes: 0.9203615982819263


In [18]:
# 100 principais features associadas a smartphones
feature_names = np.array(vect.get_feature_names())
coefs_sorted = modelos['logistic_regression'].coef_[0].argsort()
feature_names[coefs_sorted][-100:]

array(['1sim', '6gb', 'mini', 'xt1676', 'j5', 'g6', 'indigo', 'j810m',
       'orig', '20mp', 'câmera', 'g570m', 'j8', 'g5', 'duro', 'termica',
       'queda', 'na', 'xperia', 'pronta', 'leitor', 'grátis', '3gb',
       'entrega', 'no', 'grafite', 'pocket', 'a1', '2017', 'digital',
       'test', 'nuevo', 'ofertar', 'item', 'sim', 'frete', 'processador',
       '32', 's5', 'duos', 'promoção', '4gb', 'ano', 'android', 'tv',
       'j400m', 'j4', 's60', 'caterpillar', 'cat', 'k10', 'play', 'redmi',
       'e5', 'global', 'chip', 'apple', 'samsung', 'desbloqueado',
       '128gb', 'lg', 'xt1726', 'mi', 'fiscal', 'ram', '8mp', 'nf',
       'garantia', 'prime', 'nota', '8gb', 'moto', '3g', 'gb', 'j7',
       'ferrari', 'anatel', 'novo', 'vitrine', 'dual', 'nextel',
       'dourado', '4g', 'lacrado', 'g800h', '16gb', 'tela', 'plus',
       'original', 'xiaomi', '256gb', 'celular', 's9', 'galaxy',
       'motorola', 'smartphone', 'i867', 'iphone', '32gb', '64gb'],
      dtype='<U23')

In [19]:
# 100 principais features associadas a não-smartphones
feature_names[coefs_sorted][:100]

array(['capa', 'bumper', 'bracadeira', 'para', 'película', 'vidro', 'tab',
       'rosa', 'kit', 'watch', 'tablet', 'tpu', 'p355m', 'azul',
       'transparente', 'vermelho', 'pc', 'de', 'bumber', 's3', '5mp',
       'fonte', 'series', '38mm', 'pelicula', 'em', 'protetora',
       'shampoo', 'preta', 'único', 'capapelicula', 'pingente', '4s',
       'ero', 'laço', 'cabo', 'papel', 'do', 'silver', 'nike', 's6102',
       's4', 'samba', 'jr', 'q53', 'carregador', 'flip', 'ml', 'blu',
       'cm', 'ciabrink', 'premium', 'dados', 'couro', 'séries', '42mm',
       'core', 'pulseira', 'amarelo', 'ghz', 'violão', 'sm', 'infantil',
       'temperado', 'litocart', 'branco', 'mirror', 'usb', 'black',
       'adesivo', 'madeira', 'espelhamento', 'au906', 'cast', 'esportiva',
       'batom', 'verde', 'micro', 'vgn', 'veicular', 'adaptador',
       'braçadeira', 'idea', 'gps', '5c', 'tracta', 'whey', 'exclusiva',
       'personalizada', 'branca', 'ddr3', 'com', 'suporte', 'coração',
       'botão',

# Parte 4: Classificação
Aplicação do melhor modelo encontrado e salvamento do resultado.

In [20]:
data = pd.read_csv('data/data_estag_ds.tsv', sep='\t')

In [21]:
data.head(10)

,ID,TITLE
0,1041354,Acessório T - Jean Bag For Girls para DS Lite
1,1041782,Carrinho de Bebê Berço-Passeio - Pegasus Pink ...
2,1041834,Carrinho de Bebê para Gêmeos Berço-Passeio - T...
3,1042568,Car Center - Calesita
4,1042584,Donka Trem com Som - Calesita
5,1043696,Interfone HDL AZ-01 Branco
6,1940486,Smartphone LG Optimus L7 II Dual P716 Preto co...
7,2277815,Smartphone Samsung Galaxy S4 Mini Duos Preto c...
8,2961097,Smartphone Samsung Galaxy S5 SM-G900M Branco ...
9,2961099,Smartphone Samsung Galaxy S5 SM-G900M Dourado...


In [22]:
vect = CountVectorizer(stop_words=pt_stopwords)
X = CountVectorizer().fit_transform(data['TITLE'])
y = modelos['logistic_regression'].predict(X)

In [23]:
data['SMARTPHONE'] = y

In [24]:
data.to_csv('data/resultados.tsv', sep='\t')